In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import tensorflow as tf
from sklearn.utils import shuffle
import jieba
import re
import time
import collections
import os

In [3]:
question = open("question.csv",encoding='utf-8')

answer = open("answer.csv",encoding='utf-8')

all_train_id = pd.read_table("train_candidates.txt",sep=',',dtype=str,nrows = None)

In [4]:
# txt去重，太慢了
del all_train_id["neg_ans_id"]
all_train_id.drop_duplicates(inplace=True)
all_train_id.reset_index(drop=True, inplace=True)
train_id = all_train_id[:1000]

In [5]:
# 简单处理一下question/answer每句话
def sentence_precessing(text):
    
    text = text.lower().strip()
    
    text = re.sub(r'["cm"]+', "厘米", text)
    text = re.sub(r"([。])", r".", text)
    text = re.sub(r"[a-zA-Z]", "",text)
    text = re.sub(r"([？?!！，¿])", r",", text)
    text = re.sub(r"([‘’“”\"\"''])", r"", text)
    text = re.sub(r"[-()（）\#/@;:<>{}`+=~|]", "", text)
    text = re.sub(r'[" "]+', "", text)
    
    return text

In [6]:
question_dict = {}
question.readline()
for line in question:
    _line = line.strip('\n').split(',')
    question_dict[_line[0]] = " ".join(jieba.cut(sentence_precessing(_line[1])))

answer_dict = {}
answer.readline()
for line in answer:
    _line = line.strip('\n').split(',')
    answer_dict[_line[0]] = " ".join(jieba.cut(sentence_precessing(_line[2])))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\leo\AppData\Local\Temp\jieba.cache
Loading model cost 0.869 seconds.
Prefix dict has been built succesfully.


In [9]:
train_questions = []
train_answers = []
for question,answer in zip(train_id["question_id"],train_id["pos_ans_id"]):
    train_questions.append(question_dict[question])
    train_answers.append(answer_dict[answer])

In [7]:
def build_dataset(words, n_words, atleast=5):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [10]:
concat_from = ' '.join(train_questions).split()
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])
print('filtered vocab size:',len(dictionary_from))
print("% of vocab used: {}%".format(round(len(dictionary_from)/vocabulary_size_from,4)*100))

vocab from size: 6114
Most common words [(',', 8720), ('了', 2124), ('的', 1943), ('.', 1633), ('我', 1478), ('是', 1291)]
Sample data [213, 45, 22, 0, 42, 134, 11, 4, 219, 14] ['不是', '说', '做', 'PAD', '宝宝', '不好', '吗', ',', '那', '怀孕']
filtered vocab size: 2014
% of vocab used: 32.940000000000005%


In [11]:
concat_to = ' '.join(train_answers).split()
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab from size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])
print('filtered vocab size:',len(dictionary_to))
print("% of vocab used: {}%".format(round(len(dictionary_to)/vocabulary_size_to,4)*100))

vocab from size: 12494
Most common words [(',', 16323), ('的', 9561), ('.', 7112), ('、', 3276), ('是', 3101), ('治疗', 2283)]
Sample data [271, 175, 0, 361, 17, 8, 256, 5, 308, 27] ['超', '属于', 'PAD', '经常', '检查', '是', '不好', '的', '而且', '也']
filtered vocab size: 3320
% of vocab used: 26.57%


In [12]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [16]:
for i in range(len(train_answers)):
    train_answers[i] += ' EOS'

In [17]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

In [18]:
X = str_idx(train_questions, dictionary_from)
Y = str_idx(train_answers, dictionary_to)
# X_test = str_idx(question_test, dictionary_from)
# Y_test = str_idx(answer_test, dictionary_from)

In [19]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [27]:
from tensorflow.python.util import nest
from tensorflow.python.layers.core import Dense

def gnmt_residual_fn(inputs, outputs):
    def split_input(inp, out):
        out_dim = out.get_shape().as_list()[-1]
        inp_dim = inp.get_shape().as_list()[-1]
        return tf.split(inp, [out_dim, inp_dim - out_dim], axis=-1)
    actual_inputs, _ = nest.map_structure(split_input, inputs, outputs)

    def assert_shape_match(inp, out):
        inp.get_shape().assert_is_compatible_with(out.get_shape())
    nest.assert_same_structure(actual_inputs, outputs)
    nest.map_structure(assert_shape_match, actual_inputs, outputs)
    return nest.map_structure(lambda inp, out: inp + out, actual_inputs, outputs)

class GNMTAttentionMultiCell(tf.nn.rnn_cell.MultiRNNCell):

    def __init__(self, attention_cell, cells, use_new_attention=True):
        cells = [attention_cell] + cells
        self.use_new_attention = use_new_attention
        super(GNMTAttentionMultiCell, self).__init__(
            cells, state_is_tuple=True)

    def __call__(self, inputs, state, scope=None):
        """Run the cell with bottom layer's attention copied to all upper layers."""
        if not nest.is_sequence(state):
            raise ValueError(
                "Expected state to be a tuple of length %d, but received: %s"
                % (len(self.state_size), state))

        with tf.variable_scope(scope or "multi_rnn_cell"):
            new_states = []

            with tf.variable_scope("cell_0_attention"):
                attention_cell = self._cells[0]
                attention_state = state[0]
                cur_inp, new_attention_state = attention_cell(
                    inputs, attention_state)
                new_states.append(new_attention_state)

            for i in range(1, len(self._cells)):
                with tf.variable_scope("cell_%d" % i):
                    cell = self._cells[i]
                    cur_state = state[i]

                    if self.use_new_attention:
                        cur_inp = tf.concat(
                            [cur_inp, new_attention_state.attention], -1)
                    else:
                        cur_inp = tf.concat(
                            [cur_inp, attention_state.attention], -1)

                    cur_inp, new_state = cell(cur_inp, cur_state)
                    new_states.append(new_state)

        return cur_inp, tuple(new_states)

class Chatbot:
    def __init__(self, size_layer, num_layers, embedded_size,
                 from_dict_size, to_dict_size, learning_rate, beam_width = 15):
        
        def cells(size,reuse=False):
            return tf.nn.rnn_cell.GRUCell(size,reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        encoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embeddings = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        decoder_embedded = tf.nn.embedding_lookup(decoder_embeddings, decoder_input)
        
        num_residual_layer = num_layers - 2
        num_bi_layer = 1
        num_ui_layer = num_layers - num_bi_layer

        for n in range(num_bi_layer):
            (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cells(size_layer),
                cell_bw = cells(size_layer),
                inputs = encoder_embedded,
                sequence_length = self.X_seq_len,
                dtype = tf.float32,
                scope = 'bidirectional_rnn_%d'%(n))
            encoder_embedded = tf.concat((out_fw, out_bw), 2)
        
        gru_cells = tf.nn.rnn_cell.MultiRNNCell([cells(size_layer) for _ in range(num_ui_layer)])
        encoder_outputs, encoder_state = tf.nn.dynamic_rnn(
                gru_cells,
                encoder_embedded,
                dtype=tf.float32,
                sequence_length=self.X_seq_len)
        
        encoder_state = (state_bw,) + (
                (encoder_state,) if num_ui_layer == 1 else encoder_state)
        
        decoder_cells = []
        for n in range(num_layers):
            cell = cells(size_layer)
            if (n >= num_layers - num_residual_layer):
                cell = tf.nn.rnn_cell.ResidualWrapper(cell, residual_fn = gnmt_residual_fn)
            decoder_cells.append(cell)
        attention_cell = decoder_cells.pop(0)
        to_dense = tf.layers.Dense(to_dict_size)
        
        with tf.variable_scope('decode'):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                num_units = size_layer, 
                memory = encoder_outputs,
                memory_sequence_length = self.X_seq_len)
            att_cell = tf.contrib.seq2seq.AttentionWrapper(
                cell = attention_cell,
                attention_mechanism = attention_mechanism,
                attention_layer_size = None,
                alignment_history = True,
                output_attention = False)
            gcell = GNMTAttentionMultiCell(att_cell, decoder_cells)
            
            self.initial_state = tuple(
                zs.clone(cell_state=es)
                if isinstance(zs, tf.contrib.seq2seq.AttentionWrapperState) else es
                for zs, es in zip(
                    gcell.zero_state(batch_size, dtype=tf.float32), encoder_state))
            
            training_helper = tf.contrib.seq2seq.TrainingHelper(
                decoder_embedded,
                self.Y_seq_len,
                time_major = False
            )
            training_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = gcell,
                helper = training_helper,
                initial_state = self.initial_state,
                output_layer = to_dense)
            training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = training_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.Y_seq_len))
            self.training_logits = training_decoder_output.rnn_output
            
        with tf.variable_scope('decode', reuse=True):
            encoder_out_tiled = tf.contrib.seq2seq.tile_batch(encoder_outputs, beam_width)
            encoder_state_tiled = tf.contrib.seq2seq.tile_batch(encoder_state, beam_width)
            X_seq_len_tiled = tf.contrib.seq2seq.tile_batch(self.X_seq_len, beam_width)
            
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                num_units = size_layer, 
                memory = encoder_out_tiled,
                memory_sequence_length = X_seq_len_tiled)
            att_cell = tf.contrib.seq2seq.AttentionWrapper(
                cell = attention_cell,
                attention_mechanism = attention_mechanism,
                attention_layer_size = None,
                alignment_history = False,
                output_attention = False)
            gcell = GNMTAttentionMultiCell(att_cell, decoder_cells)
            
            self.initial_state = tuple(
                zs.clone(cell_state=es)
                if isinstance(zs, tf.contrib.seq2seq.AttentionWrapperState) else es
                for zs, es in zip(
                    gcell.zero_state(batch_size * beam_width, dtype=tf.float32), encoder_state_tiled))
            
            predicting_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = gcell,
                embedding = decoder_embeddings,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS,
                initial_state = self.initial_state,
                beam_width = beam_width,
                output_layer = to_dense,
                length_penalty_weight = 0.0)
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = False,
                maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
            self.predicting_ids = predicting_decoder_output.predicted_ids[:, :, 0]
            
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [28]:
size_layer = 128
num_layers = 4
embedded_size = 128
learning_rate = 0.001
batch_size = 16
epoch = 20

In [29]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Chatbot(size_layer, num_layers, embedded_size, len(dictionary_from), 
                len(dictionary_to), learning_rate,batch_size)
sess.run(tf.global_variables_initializer())

C:\Users\leo\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [31]:
for i in range(epoch):
    total_loss, total_accuracy = 0, 0
    X, Y = shuffle(X, Y)
    for k in range(0, len(train_questions), batch_size):
        index = min(k + batch_size, len(train_questions))
        batch_x, seq_x = pad_sentence_batch(X[k: index], PAD)
        batch_y, seq_y = pad_sentence_batch(Y[k: index], PAD)
        predicted, accuracy, loss, _ = sess.run([model.predicting_ids,
                                      model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
        total_loss += loss
        total_accuracy += accuracy
#         print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1, loss, accuracy))
    total_loss /= (len(train_questions) / batch_size)
    total_accuracy /= (len(train_questions) / batch_size)
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1, total_loss, total_accuracy))

KeyboardInterrupt: 

In [23]:
for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
QUESTION: watch the news
REAL ANSWER: nah i musta missed it
PREDICTED ANSWER: nah i musta missed 

row 2
QUESTION: i love you elaine
REAL ANSWER: i love you
PREDICTED ANSWER: i i 

row 3
QUESTION: daddy no
REAL ANSWER: just for a minute
PREDICTED ANSWER: just for 

row 4
QUESTION: good who
REAL ANSWER: mel gordon
PREDICTED ANSWER: mel gordon 



In [24]:
batch_x, seq_x = pad_sentence_batch(X_test[:batch_size], PAD)
batch_y, seq_y = pad_sentence_batch(Y_test[:batch_size], PAD)
predicted = sess.run(model.predicting_ids, feed_dict={model.X:batch_x})

for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
QUESTION: but david
REAL ANSWER: is here that
PREDICTED ANSWER: i i divorced 

row 2
QUESTION: hopeless it is hopeless
REAL ANSWER: tell ballet then back
PREDICTED ANSWER: was was the 

row 3
QUESTION: miss price
REAL ANSWER: yes learning
PREDICTED ANSWER: alex 

row 4
QUESTION: mr kessler wake up please
REAL ANSWER: is here are
PREDICTED ANSWER: who 

row 5
QUESTION: there were witnesses
REAL ANSWER: why she out
PREDICTED ANSWER: who it it 

row 6
QUESTION: what about it
REAL ANSWER: not you are
PREDICTED ANSWER: ido her 

row 7
QUESTION: go on ask them
REAL ANSWER: i just home
PREDICTED ANSWER: on on on 

row 8
QUESTION: beware the moon
REAL ANSWER: seen hi is he
PREDICTED ANSWER: who what the 

row 9
QUESTION: did you hear that
REAL ANSWER: is down what
PREDICTED ANSWER: how did 

row 10
QUESTION: i heard that
REAL ANSWER: it here not
PREDICTED ANSWER: and then 

row 11
QUESTION: the hound of the baskervilles
REAL ANSWER: heard
PREDICTED ANSWER: thirtyfour time 

row 12
QUESTI